## [세 가지 질문으로 살펴본 우리나라의 민원](https://soprize.so/question/81)

* 국민권익위원회 민원빅데이터 분석정보 API 활용
* 분석연도: 2005년 - 2020년 

### 1. [누가?](#1)
- 어떤 성별, 연령의 사람들이 민원을 제기했을까요?

### 2. [언제?](#2)
- 어느 계절의, 하루의 어느 시간대에 사람들이 민원을 제기했을까요?

### 3. [무엇을?](#3)
- 민원인은 어떤 내용을 민원으로 제기했을까요?

In [10]:
import requests
import ast
import pandas as pd
import numpy as np
from collections import defaultdict
import plotly.graph_objects as go
import plotly.express as px

In [19]:
# API 신청: https://www.data.go.kr/dataset/15040459/openapi.do
KEY = YOUR_API_KEY

### <a id='1'> </a>1. 누가?
### 1.1. 성별에 따른 민원 건 수

In [49]:
def annual_sum(text):
    total = 0
    for word in ast.literal_eval(result.text):
        total += int(word['hits'])
    return total

In [50]:
# 연도별 여성의 민원 건 수 
woman_dict = {}

startdate = '20050101000000'.format(year)
enddate = '20201231235959'.format(year)

url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minStaticsInfo2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom={}&dateTo={}&topN=10000&period=YEARLY&sortBy=NAME&sortOrder=false&sex=w".format(KEY, startdate, enddate)
result = requests.get(url) 

woman_dict[year] = annual_sum(result.text)

In [51]:
woman_dict

{'2005': 5977,
 '2006': 112864,
 '2007': 163226,
 '2008': 174941,
 '2009': 196699,
 '2010': 249006,
 '2011': 336500,
 '2012': 365875,
 '2013': 378455,
 '2014': 304919,
 '2015': 254886,
 '2016': 276269,
 '2017': 476205,
 '2018': 1340390,
 '2019': 2284858,
 '2020': 1541805}

In [52]:
# 연도별 남성의 민원 건 수 
man_dict = {}

for year in range(5, 21):
    year = '20' + str(year).zfill(2)
    startdate = '{}0101000000'.format(year)
    enddate = '{}1231235959'.format(year)

    url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minStaticsInfo2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom={}&dateTo={}&topN=10000&period=MONTHLY&sortBy=NAME&sortOrder=false&sex=m".format(KEY, startdate, enddate)
    result = requests.get(url) 

    man_dict[year] = annual_sum(result.text)

In [53]:
man_dict

{'2005': 13674,
 '2006': 266833,
 '2007': 342863,
 '2008': 407353,
 '2009': 445179,
 '2010': 522582,
 '2011': 698734,
 '2012': 823789,
 '2013': 1019094,
 '2014': 1032919,
 '2015': 1052735,
 '2016': 1155717,
 '2017': 1046841,
 '2018': 1851700,
 '2019': 2981675,
 '2020': 2146730}

In [48]:
years = list(woman_dict.keys())

fig = go.Figure(data=[
    go.Bar(name='여성', x=years, y=list(woman_dict.values())),
    go.Bar(name='남성', x=years, y=list(man_dict.values()))
])
fig.update_layout(barmode='group', title={'text': '연도와 성별에 따른 민원 수', 'y':0.86, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.update_xaxes(type='category', tickangle=20)
fig.show()

### 1.2. 연령대에 따른 민원 건 수

In [76]:
age_dict = {}

In [77]:
def total_sum(text):
    total = 0
    for word in ast.literal_eval(result.text):
        total += int(word['hits'])
    return total

In [78]:
# 연령별 민원 건 수 
for i in range(1,10):
    age = str(i * 10)
    startdate = '20050101000000'
    enddate = '20201231235959'

    url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minStaticsInfo2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom={}&dateTo={}&topN=10000&period=YEARLY&sortBy=NAME&sortOrder=false&age={}".format(KEY, startdate, enddate, age)
    result = requests.get(url) 
    age_dict[age] = total_sum(result.text)

In [79]:
age_dict

{'10': 450280,
 '20': 3597699,
 '30': 9383123,
 '40': 6847070,
 '50': 3357066,
 '60': 1055828,
 '70': 155461,
 '80': 23599,
 '90': 1950}

In [49]:
x = [str(age) + '대' for age in list(age_dict.keys())]
y = list(age_dict.values())
text = ['450K', '3M', '9M', '6M', '3M', '1M', '155K', '23K', '1K']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=text,
            textposition='auto',
        )])
fig.update_layout(title={'text': '연령대에 따른 2005년-2020년 민원 수', 'y':0.86, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

### 1.3. 거주지에 따른 민원 건 수

In [92]:
url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minMrfacetInfo2?serviceKey={}&topN=10&sortBy=NAME&sortOrder=false&dateFrom=20050101&dateTo=20201231&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin".format(KEY)
result = requests.get(url) 

df = pd.DataFrame()

count = []
regions = []
for region in ast.literal_eval(result.text):
    count.append(int(region['hits']))
    regions.append(region['label'])

df['region'] = regions
df['count'] = count
df

,region,count
0,경기도,14067457
1,NONE,14013331
2,서울특별시,7732329
3,인천광역시,2512308
4,부산광역시,1930478
5,경상남도,1704029
6,경상북도,1352876
7,대전광역시,1242615
8,대구광역시,1224752
9,충청북도,665053


In [107]:
# 인구 수: https://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=1007
df = pd.read_csv('region.csv')[['region', 'count', 'population']]
df

,region,count,population
0,경기도,14067457,13405000
1,서울특별시,7732329,9602000
2,인천광역시,2512308,2951000
3,부산광역시,1930478,3344000
4,경상남도,1704029,3350000
5,경상북도,1352876,2655000
6,대전광역시,1242615,1500000
7,대구광역시,1224752,2419000
8,충청북도,665053,1632000
9,전라북도,576685,1792000


In [111]:
df['ratio'] = df.apply(lambda x: x[1]/x[2], axis=1)
df = df.sort_values(by=['ratio'], ascending=False)
df

,region,count,population,ratio
0,경기도,14067457,13405000,1.049419
2,인천광역시,2512308,2951000,0.851341
6,대전광역시,1242615,1500000,0.828410
1,서울특별시,7732329,9602000,0.805283
3,부산광역시,1930478,3344000,0.577296
5,경상북도,1352876,2655000,0.509558
4,경상남도,1704029,3350000,0.508665
7,대구광역시,1224752,2419000,0.506305
8,충청북도,665053,1632000,0.407508
10,광주광역시,526991,1488000,0.354161


In [116]:
x = df.region.tolist()
y = df.ratio.tolist()
text = [str(val)[:4] for val in y]

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=text,
            textfont_size=10,
            textposition='auto',
        )])
fig.update_layout(title={'text': '거주지에 따른 2005년-2020년 총 민원 수와 인구 수의 비율', 'y':0.86, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

### <a id='2'> </a>2. 언제?
### 2.1. 계절에 따른 민원 건 수 

In [ ]:
startdate = '20050101000000'.format(year)
enddate = '20201231235959'.format(year)

url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minStaticsInfo2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom={}&dateTo={}&topN=10000&period=MONTHLY&sortBy=NAME&sortOrder=false".format(KEY, startdate, enddate)
result = requests.get(url) 

In [83]:
season_dict = defaultdict(int)

for x in ast.literal_eval(result.text):
    # 봄
    if x['label'][4:] in ['0301', '0401', '0501']:
        season_dict['spring'] += x['hits']
    # 여름
    if x['label'][4:] in ['0601', '0701', '0801']:
        season_dict['summer'] += x['hits']
    # 가을
    if x['label'][4:] in ['0901', '1001', '1101']:
        season_dict['autumn'] += x['hits']
    # 겨울
    if x['label'][4:] in ['1201', '0101', '0201']:
        season_dict['winter'] += x['hits']

In [84]:
season_dict

defaultdict(int,
            {'winter': 11340395,
             'spring': 12918113,
             'summer': 15278211,
             'autumn': 14447979})

In [38]:
season_dict = {'봄': 12918113,
               '여름': 15278211,
               '가을': 14447979,
               '겨울': 11340395}

In [50]:
x = list(season_dict.keys())
y = list(season_dict.values())
text = ['12.9M', '15.2M', '14.4M', '11.3M']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=text,
            textposition='auto',
        )])
fig.update_layout(title={'text': '계절에 따른 2005년-2020년 민원 수', 'y':0.86, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

### 2.2. 시간대에 따른 민원 건 수

In [98]:
time_dict = {}

In [99]:
# 아침 
morning = 0
for month in range(1,13):
    for date in range(1,32):
        month = str(month).zfill(2)
        date = str(date).zfill(2)
        day = month + date
        try:
            startdate = '2020{}060000'.format(day)
            enddate = '2020{}115959'.format(day)

            url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minStaticsInfo2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom={}&dateTo={}&topN=10000&period=DAILY&sortBy=NAME&sortOrder=false".format(KEY, startdate, enddate)
            result = requests.get(url) 

            morning += int(ast.literal_eval(result.text)[0]['hits'])
            
        except:
            pass
        
time_dict['morning'] = morning

In [100]:
# 점심
noon = 0
for month in range(1,13):
    for date in range(1,32):
        month = str(month).zfill(2)
        date = str(date).zfill(2)
        day = month + date
        try:
            startdate = '2020{}120000'.format(day)
            enddate = '2020{}175959'.format(day)

            url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minStaticsInfo2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom={}&dateTo={}&topN=10000&period=DAILY&sortBy=NAME&sortOrder=false".format(KEY, startdate, enddate)
            result = requests.get(url) 

            noon += int(ast.literal_eval(result.text)[0]['hits'])
            
        except:
            pass
        
time_dict['noon'] = noon

In [101]:
# 저녁
evening = 0
for month in range(1,13):
    for date in range(1,32):
        month = str(month).zfill(2)
        date = str(date).zfill(2)
        day = month + date
        try:
            startdate = '2020{}180000'.format(day)
            enddate = '2020{}235959'.format(day)

            url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minStaticsInfo2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom={}&dateTo={}&topN=10000&period=DAILY&sortBy=NAME&sortOrder=false".format(KEY, startdate, enddate)
            result = requests.get(url) 

            noon += int(ast.literal_eval(result.text)[0]['hits'])
            
        except:
            pass
        
time_dict['evening'] = evening

In [102]:
time_dict

{'morning': 3419266, 'noon': 4819010, 'evening': 0}

In [57]:
time_dict = {'아침 (06시-12시)': 3419266, '점심 (12시-18시)': 4819010, '저녁 (18시-24시)': 0}

In [58]:
x = list(time_dict.keys())
y = list(time_dict.values())
text = ['3.4M', '4.8M', '0']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=text,
            textposition='auto',
        )])
fig.update_layout(title={'text': '시간대에 따른 2020년 민원 수', 'y':0.86, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

### <a id='3'> </a> 3. 무엇을?

In [37]:
def to_df(text, year):
    df = pd.DataFrame()
    labels = []
    values = []
    for word in ast.literal_eval(result.text):
        labels.append(word['label'])
        values.append(int(word['value'].split('.')[0]))

    df[year + '_word'] = labels
    df[year + '_count'] = values
    return df

In [43]:
# 연도별 민원 키워드
df = pd.DataFrame()
for year in range(5, 21):
    year = '20' + str(year).zfill(2)
    startdate = '{}0101'.format(year)
    enddate = '{}1231'.format(year)
    url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minTopNKeyword2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom={}&dateTo={}&resultCount=50&searchCount=1000000".format(KEY, startdate, enddate)

    result = requests.get(url) 
    temp = to_df(result.text, str(year))
    df = pd.concat([df, temp], axis=1)

In [44]:
df

,2005_word,2005_count,2006_word,2006_count,2007_word,2007_count,2008_word,2008_count,2009_word,2009_count,...,2016_word,2016_count,2017_word,2017_count,2018_word,2018_count,2019_word,2019_count,2020_word,2020_count
0,동탄,1436,경찰서,844,경찰서,568,위원회,466,위원회,246,...,대구,824,광주,756,광주,1202,휴대폰,1504,수정구청,930
1,경찰서,432,부산,520,피해자,320,면사무소,360,경찰서,184,...,정류장,478,대구,590,구조물,742,분당구청,996,분당구청,868
2,위원회,272,핸드폰,462,청주,276,청각 장애,352,가해자,112,...,대구시,388,정류장,312,도시철,742,분당,978,분당,810
3,동탄 신도시,270,피해자,384,핸드폰,242,머슴들,238,부대장,108,...,광주,278,민원실,218,구조물 벽면,676,수정구청,938,휴대폰,792
4,피해자,258,가해자,380,위원회,214,경찰서,212,소유자,102,...,매연신고,178,주차장,216,구조물 벽면 녹물제거,676,성남,868,성남,528
5,중학교,238,전화번호,256,가해자,198,경기도,210,대법원,98,...,주정차,176,종합민원실,214,녹물제거,676,중원구청,616,가로등,400
6,입주민,236,휴대폰,172,재개발,194,공직자,200,서울,96,...,비서실,158,입주민,156,대구,500,과천,570,주정차,392
7,사립고,230,경찰청,146,운전자,190,장애인,198,뇌파,90,...,주차장,158,대구시,148,정류장,228,정류장,348,코로나,226
8,교육청,224,주차장,142,전화번호,158,보험료,196,운전병,90,...,직소민원,158,가로등,146,광주광역시 북구,220,현수막,340,정류장,206
9,고등학교,222,교통사고,136,범칙금,150,임대료,188,청와대,86,...,부산,150,자갈마당,144,대구시,214,주정차,296,노점상,194


In [45]:
df.to_csv('연도별_키워드.csv', index=False, encoding='utf-8')

In [109]:
domains = ['경찰', '과학기술', '관광', '교육', '교통', '국방', '기타', '노동', '농림·축산', '도로', '도시', '문화', '방송통신', '법무', '보건', '복지', '사법', '산업·통상', '세무', '수자원', '외무통일', '우정', '재정', '주택·건축', '체육', '해양수산', '행정·안전', '환경']

In [113]:
df = pd.DataFrame(columns = ['domain'] + ['20' + str(i).zfill(2) for i in range(5,21)])
count = 0

# 분야별 연도에 따른 민원 건 수 
for domain in domains:
    url = 'http://apis.data.go.kr/1140100/minAnalsInfoView2/minStaticsInfo2?serviceKey={}&target=pttn,dfpt,saeol,prpl,twitter,qna,qna_origin&dateFrom=20050101000000&dateTo=20201231235959&topN=10000&period=YEARLY&sortBy=NAME&sortOrder=false&classificationMulti={}'.format(KEY, domain)
    result = requests.get(url) 
    
    vals = [domain]
    for x in ast.literal_eval(result.text):
        vals.append(int(x['hits']))
    
    df.loc[count] = vals
    count += 1 
        

In [114]:
df

,domain,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,경찰,1753,21690,29530,16704,19500,72580,95876,155087,228194,378628,1180283,1974021,2103621,1382058,4225464,4867080
1,과학기술,1585,21572,33656,8291,9722,36301,45258,53553,70103,55661,56331,62157,63235,62463,70287,82008
2,관광,717,4240,6238,4222,7030,21202,24462,25187,27945,32395,52824,58978,73955,103199,107333,112649
3,교육,3059,26827,42980,8986,12917,47640,66403,72135,94730,92466,85153,109757,236651,193884,196379,578648
4,교통,1344,18905,15766,6767,10281,47761,61994,77989,94122,116026,251327,321665,364051,400905,1055106,1054370
5,국방,1061,12293,19434,6101,7965,47288,97087,88620,87211,83752,79272,73542,77555,78811,86910,159210
6,기타,1954,91248,123691,63304,68723,82425,93763,93663,108023,120744,187307,233519,260183,386244,501897,608720
7,노동,1494,25739,54008,6442,8475,78744,96435,96215,96021,89319,82743,78497,108415,137538,153294,195610
8,농림·축산,798,11009,14048,5248,6207,25361,28705,27017,31932,34069,42649,45306,56822,63056,64639,82472
9,도로,682,12377,15538,5861,8332,32690,37719,43640,50118,57300,323668,444740,511674,345353,808703,715671


In [115]:
df.to_csv('분야별_연도별_민원건수.csv', index=False, encoding='utf-8')

In [67]:
for col in df.columns:
    if col == 'domain':
        continue
    df[col] = df[col].apply(lambda x: (x/sum(df[col].tolist()))*100)
    
df.head()

,domain,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,경찰,4.545455,4.142016,4.165568,6.308943,6.335921,7.294018,7.659712,10.995920,13.448040,20.225822,33.132946,40.766236,38.157367,27.605444,38.307775,38.545169
1,과학기술,4.109838,4.119483,4.747590,3.131433,3.158863,3.648114,3.615746,3.796994,4.131344,2.973339,1.581326,1.283627,1.147013,1.247646,0.637217,0.649468
2,관광,1.859151,0.809689,0.879946,1.594610,2.284181,2.130722,1.954315,1.785799,1.646868,1.730499,1.482877,1.217976,1.341462,2.061313,0.973074,0.892131
3,교육,7.931857,5.123000,6.062855,3.393927,4.196979,4.787641,5.305059,5.114488,5.582674,4.939415,2.390418,2.266632,4.292588,3.872670,1.780359,4.582642
4,교통,3.484935,3.610181,2.223987,2.555832,3.340492,4.799801,4.952816,5.529546,5.546843,6.197960,7.055260,6.642823,6.603484,8.007740,9.565521,8.350154


In [105]:
title = '연도와 분야에 따른 민원 수'
labels = df.domain.tolist()
colors = ['rgb(0,0,0)', 'rgb(87,87,87)', 'rgb(173,35,35)', 'rgb(42,75,215)',\
'rgb(29,105,20)', 'rgb(129,74,25)', 'rgb(129,38,192)', 'rgb(160,160,160)',\
'rgb(129,197,122)', 'rgb(157,175,255)', 'rgb(41,208,208)', 'rgb(255,146,51)',\
'rgb(255,238,51)', 'rgb(233,222,187)', 'rgb(255,205,243)', 'rgb(255,255,255)']

x_data = np.vstack((np.arange(2005, 2021),)*16)

y_data = []
for row in range(len(df)):
    y_data.append(list(df.iloc[row, :][1:]))


fig = go.Figure()

for i in range(16):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        showlegend=False,
        marker=dict(color=colors[i])
    ))

    
fig.update_layout(title={'text': '연도와 분야에 따른 민원 수', 'y':0.86, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                   yaxis_title='특정 분야 민원의 비율 (%)')

fig.show()

### 3.1. 피해자, 가해자 (2005년-2014년)

In [103]:
# 피해자
url = 'http://apis.data.go.kr/1140100/minAnalsInfoView2/minWdcloudInfo2?serviceKey={}&target=pttn,dfpt,saeol&classification=&searchword=피해자&dateFrom=20050101&dateTo=20141231&resultCount=10&searchCount=1000'.format(KEY)
result = requests.get(url) 
result.text

'[{"label":"가해자","value":369.52762},{"label":"피의자","value":128.1394},{"label":"경찰서","value":103.55319},{"label":"보험사","value":71.02941},{"label":"교통사고","value":64.485985},{"label":"국토부","value":41.35965},{"label":"거짓말","value":33.4875},{"label":"성폭력","value":32.8125},{"label":"그린벨트","value":31.925581},{"label":"검찰청","value":28.956522},{"label":"진단서","value":28.762646},{"label":"범죄자","value":28.285715},{"label":"피고인","value":27.096775},{"label":"변호사","value":26.72889},{"label":"투기꾼","value":24.34909},{"label":"합의금","value":23.643312},{"label":"재산권","value":22.927374},{"label":"상해","value":22.834286},{"label":"담당조사관","value":21.248447},{"label":"학교폭력","value":20.903225}]'

In [ ]:
# 가해자
url = 'http://apis.data.go.kr/1140100/minAnalsInfoView2/minWdcloudInfo2?serviceKey={}&target=pttn,dfpt,saeol&classification=&searchword=가해자&dateFrom=20050101&dateTo=20141231&resultCount=10&searchCount=1000'.format(KEY)
result = requests.get(url) 
result.text

### 3.2. 주정차 (2015년-2020년)

In [ ]:
# 주정차
url = 'http://apis.data.go.kr/1140100/minAnalsInfoView2/minWdcloudInfo2?serviceKey={}&target=pttn,dfpt,saeol&classification=&searchword=주정차&dateFrom=20150101&dateTo=20201231&resultCount=10&searchCount=1000'.format(KEY)
result = requests.get(url) 
result.text

### 3.3. 전염병: 메르스(2015년)와 코로나(2020년)

In [107]:
def to_df2(text, year):
    df = pd.DataFrame()
    labels = []
    values = []
    for word in ast.literal_eval(result.text):
        labels.append(word['label'])
        values.append(word['value'])

    df[year + '_word'] = labels
    df[year + '_value'] = values
    return df

In [ ]:
# 2015년, 메르스
url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minWdcloudInfo2?serviceKey={}&target=pttn,dfpt,saeol&classification=&searchword=메르스&dateFrom=20150101&dateTo=20151231&resultCount=10&searchCount=1000".format(KEY)
result = requests.get(url) 
mers = to_df2(result.text, '메르스')

In [104]:
# 2020년, 코로나
url = "http://apis.data.go.kr/1140100/minAnalsInfoView2/minWdcloudInfo2?serviceKey={}&target=pttn,dfpt,saeol&classification=&searchword=코로나&dateFrom=20200101&dateTo=20201231&resultCount=10&searchCount=1000".format(KEY)
result = requests.get(url) 
print(result.text)

'[{"label":"과밀 학급","value":381.0},{"label":"교육부","value":232.69417},{"label":"학급 과밀","value":225.9007},{"label":"마스크 착용","value":65.25941},{"label":"마스크 지급","value":63.52129},{"label":"발열체크","value":61.75494},{"label":"확진자","value":59.15929},{"label":"모라동 주공","value":59.142857},{"label":"부산광역시 사상","value":55.2},{"label":"증거 자료","value":55.2},{"label":"주민등록번호","value":53.98107},{"label":"보건소","value":33.701206},{"label":"외국인","value":13.448276},{"label":"부산","value":11.870968},{"label":"수도권","value":11.555555},{"label":"자영업자","value":11.538462},{"label":"가능여부","value":10.91358},{"label":"선별진료소","value":10.08},{"label":"우리나라","value":8.75},{"label":"접촉자","value":8.275862}]'

In [108]:
corona = to_df2(result.text, '코로나')
corona

,코로나_word,코로나_value
0,과밀 학급,381.000000
1,교육부,232.694170
2,학급 과밀,225.900700
3,마스크 착용,65.259410
4,마스크 지급,63.521290
5,발열체크,61.754940
6,확진자,59.159290
7,모라동 주공,59.142857
8,부산광역시 사상,55.200000
9,증거 자료,55.200000
